In [1]:
#pip install transformers

In [2]:
#!pip install opencv-python

In [3]:
#a estratégia é:
#1)experimentar DenseNet121-res224-all transfer learning (img) + BlueBERT feature extraction (text) -> denselayer
#se a anterior não funcionar (não foi preciso!!!)
#2)experimentar DenseNet121-res224-chex ou mimic_ch transfer learning (img) + BlueBERT feature extraction (text) -> denselayer

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import AutoTokenizer, AutoModel
from torchvision import datasets, models, transforms
import torchvision.transforms as transforms
import torchxrayvision as xrv
import time
from collections import Counter
import os
import skimage
from PIL import Image
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import cv2

In [6]:
#!git clone https://github.com/ieee8023/covid-chestxray-dataset

In [7]:
data = xrv.datasets.COVID19_Dataset(imgpath="covid-chestxray-dataset/images/",csvpath="covid-chestxray-dataset/metadata.csv")

In [8]:
data

{'Aspergillosis': {np.float32(0.0): 534, np.float32(1.0): 1},
 'Aspiration': {np.float32(0.0): 534, np.float32(1.0): 1},
 'Bacterial': {np.float32(0.0): 487, np.float32(1.0): 48},
 'COVID-19': {np.float32(0.0): 193, np.float32(1.0): 342},
 'Chlamydophila': {np.float32(0.0): 534, np.float32(1.0): 1},
 'Fungal': {np.float32(0.0): 512, np.float32(1.0): 23},
 'H1N1': {np.float32(0.0): 534, np.float32(1.0): 1},
 'Herpes ': {np.float32(0.0): 532, np.float32(1.0): 3},
 'Influenza': {np.float32(0.0): 531, np.float32(1.0): 4},
 'Klebsiella': {np.float32(0.0): 526, np.float32(1.0): 9},
 'Legionella': {np.float32(0.0): 526, np.float32(1.0): 9},
 'Lipoid': {np.float32(0.0): 527, np.float32(1.0): 8},
 'MERS-CoV': {np.float32(0.0): 527, np.float32(1.0): 8},
 'MRSA': {np.float32(0.0): 534, np.float32(1.0): 1},
 'Mycoplasma': {np.float32(0.0): 530, np.float32(1.0): 5},
 'No Finding': {np.float32(0.0): 520, np.float32(1.0): 15},
 'Nocardia': {np.float32(0.0): 531, np.float32(1.0): 4},
 'Pneumocystis': 

COVID19_Dataset num_samples=535 views=['PA', 'AP'] data_aug=None

In [9]:
df = data.csv

In [10]:
#print(f"Shape da imagem no índice {i}: {img.shape}")

In [11]:
def laplacian(img):
  return cv2.Laplacian(img, cv2.CV_64F).var() #aplica filtro laplaciano para detectar bordas das imagens
    #faz isso através da variância: regiões mais nítidas (i.e. bordas) têm maior variância

In [12]:
class QuadtreeNode:
    #Nó de uma Quadtree para fusão de imagens
    def __init__(self, x, y, width, height, images): #canto sup esquerdo
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.images = images  #lista de imagens de entrada
        self.children = []
        self.best_image_index = -1 #índice da imagem com o melhor foco para este nó

    def split(self, threshold_size):
        #Divide o nó em quatro filhos (/2 cada dimensão) se o tamanho for maior que o limite
        if self.width <= threshold_size or self.height <= threshold_size:
            # Se o nó é pequeno o suficiente, encontre a melhor imagem para ele
            self.find_best_focus()
            return

        half_width = self.width // 2
        half_height = self.height // 2

        #cri os quatro filhos
        self.children.append(QuadtreeNode(self.x, self.y, half_width, half_height, self.images))
        self.children.append(QuadtreeNode(self.x + half_width, self.y, self.width - half_width, half_height, self.images))
        self.children.append(QuadtreeNode(self.x, self.y + half_height, half_width, self.height - half_height, self.images))
        self.children.append(QuadtreeNode(self.x + half_width, self.y + half_height, self.width - half_width, self.height - half_height, self.images))

        #recursivamente divida os filhos
        for child in self.children:
            child.split(threshold_size)

    def find_best_focus(self):
        #encontra a imagem com a maior medida de foco (maior laplacian) para a região do nó
        max_focus = -1
        best_index = -1

        for i, img in enumerate(self.images):
            #extrai a região de interesse (ROI) da imagem
            roi = img[self.y : self.y + self.height, self.x : self.x + self.width]

            #garante que o ROI não esteja vazio antes de calcular a medida de foco
            if roi.size == 0:
                continue

            focus_measure = laplacian(roi)
            if focus_measure > max_focus:
                max_focus = focus_measure
                best_index = i
        self.best_image_index = best_index

In [13]:
def quadtree_focus_fusion(images, threshold_size=32):
    #realiza a fusão de imagens multifocais usando uma Quadtree
    if not images:
        return None

    #as imagens devem ter o mesmo tamanho
    height, width = images[0].shape

    #cria uma imagem de saída base
    fused_image = np.zeros_like(images[0])

    #adiciona o nó raiz da Quadtree
    root = QuadtreeNode(0, 0, width, height, images)
    root.split(threshold_size)

    #auxiliar para reconstruir a imagem fusionada
    def reconstruct_image(node, output_image):
        #se o nó tem uma melhor imagem definida, copia para output_image
        #se não, visita os filhos
        if node.best_image_index != -1:
            #se é um nó folha (ou um nó onde a melhor imagem foi encontrada)
            best_img_roi = node.images[node.best_image_index][node.y : node.y + node.height, node.x : node.x + node.width]
            output_image[node.y : node.y + node.height, node.x : node.x + node.width] = best_img_roi
        else:
            #se não é um nó folha, aprofunda na árvore 
            for child in node.children:
                reconstruct_image(child, output_image)

    reconstruct_image(root, fused_image)
    return fused_image

In [ ]:
#tudo isso é feito para aumentar sintéticamente o tamanho do dataset. Ele é dobrado de tamanho,
#pois preservarmos a imagem e sua versão fusionada com a blurred
#encontramos usos dessa técnica na literatura, em contextos semelhantes ao nosso.
#referencia: https://www.sciencedirect.com/science/article/abs/pii/S0010482524015920

In [14]:
def save_images_and_prepare_data(data, save_dir="covid_images_organized"):
    os.makedirs(save_dir, exist_ok=True)

    image_paths = []
    texts = []
    labels = []

    for i in range(len(data)):
        #obtem imagem (normalizada entre 0-1)
        img = data[i]["img"]
        img = np.squeeze(img)

        #converte para uint8 e PIL
        img_uint8 = (img * 255).astype(np.uint8)
        pil_img = Image.fromarray(img_uint8)

        #obtem filename do DataFrame
        filename = data.csv.iloc[i]['filename']
        if not isinstance(filename, str):  #pula se for NaN ou inválido
            continue

        #caminho completo para salvar
        filepath = os.path.join(save_dir, filename)
        pil_img.save(filepath)
        image_paths.append(filepath)

        # Texto clínico associado, string ou vazio
        clinical_note = data.csv.iloc[i]['clinical_notes']
        texts.append(clinical_note if isinstance(clinical_note, str) else "")

        # Label binária para COVID-19
        lab_series = pd.Series(dict(zip(data.pathologies, data[i]["lab"])))
        labels.append(lab_series['COVID-19'])

        #blur
        mean_filter = np.ones((3, 3), dtype=np.float32) / 9.0
        img_blurred = cv2.filter2D(img_uint8, -1, mean_filter)

        #aplica quadtree focus fusion
        fused_img = quadtree_focus_fusion([img_uint8, img_blurred], threshold_size=32)

        #salvar imagem fusionada
        filepath_fused = os.path.join(save_dir, f"fused_{filename}")
        fused_pil = Image.fromarray(fused_img)
        fused_pil.save(filepath_fused)

        image_paths.append(filepath_fused)
        texts.append(clinical_note if isinstance(clinical_note, str) else "")
        labels.append(lab_series['COVID-19'])

    return image_paths, texts, labels


In [15]:
def create_train_val_test_split(image_paths, texts, labels, test_size=0.2, val_size=0.1, random_state=42):
    trainval_img, test_img, trainval_txt, test_txt, trainval_lab, test_lab = train_test_split(
        image_paths, texts, labels, test_size=test_size, random_state=random_state, stratify=labels
    )

    val_ratio = val_size / (1 - test_size)
    train_img, val_img, train_txt, val_txt, train_lab, val_lab = train_test_split(
        trainval_img, trainval_txt, trainval_lab, test_size=val_ratio, random_state=random_state, stratify=trainval_lab
    )

    return (train_img, train_txt, train_lab), (val_img, val_txt, val_lab), (test_img, test_txt, test_lab)


In [16]:
image_transforms = {
    'train': transforms.Compose([
        transforms.Grayscale(num_output_channels=1), #xrays are b&w but densenet expects rgb
        transforms.Resize((224, 224)),
        transforms.RandomRotation(degrees=9),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(), #image (format PIL) -> tensor (format pytorch)
        #normalizar por 255 - ToTensor já faz isso automatico
        transforms.Normalize([0.5], [0.5])  
    ]),
    'val': transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])  
    ]),
    'test': transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(), 
        transforms.Normalize([0.5], [0.5])  
    ])
}

In [17]:
image_paths, texts, labels = save_images_and_prepare_data(data, save_dir="covid_images_organized")

(train_img, train_txt, train_lab), (val_img, val_txt, val_lab), (test_img, test_txt, test_lab) = create_train_val_test_split(
    image_paths, texts, labels, test_size=0.2, val_size=0.1
)

In [18]:
print(len(image_paths))

1070


In [19]:
tokenizer = AutoTokenizer.from_pretrained('bionlp/bluebert_pubmed_uncased_L-12_H-768_A-12')

In [24]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [20]:
#criando dataset multimodal, ou seja, que comparta as imagens e os textos
class MultimodalDataset(Dataset):
    def __init__(self, image_paths, texts, labels, tokenizer, transform=None):
        self.image_paths = image_paths
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        #imagem
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('L')  #abre imagem em greyscale
    
        if self.transform:
            image = self.transform(image)

        # Texto
        encoding = self.tokenizer(
            self.texts[idx],
            padding='max_length',
            truncation=True,
            max_length=128,
            return_tensors='pt'
        )
        label = torch.tensor(self.labels[idx]).long()

        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'image': image,
            'label': label
        }

In [21]:
#cria os datasets e dataloaders
train_dataset = MultimodalDataset(train_img, train_txt, train_lab, tokenizer, transform=image_transforms['train'])
val_dataset = MultimodalDataset(val_img, val_txt, val_lab, tokenizer, transform=image_transforms['val'])
test_dataset = MultimodalDataset(test_img, test_txt, test_lab, tokenizer, transform=image_transforms['test'])

train_data_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_data_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_data_loader = DataLoader(test_dataset, batch_size=24, shuffle=False)

In [22]:
#reduzimos um pouco o threshold de decisão, para favorecer a classificação como covid em casos de dúvida
#evitando falsos negativos e maximizando o recall, como queríamos
threshold = 0.475

In [25]:
#mesma função que usamos no trabalho todo, mas agora adaptada à multimodalidade
def train_and_validate_multimodal(model, loss_criterion, optimizer, epochs, dataset="multimodal",
                                  train_data_loader=train_data_loader, val_data_loader=val_data_loader,
                                  train_data_size=len(train_dataset), val_data_size=len(val_dataset), device=device):

    start = time.time()
    history = []
    best_loss = 100000.0
    best_epoch = None

    for epoch in range(epochs):
        epoch_start = time.time()

        model.train()

        train_loss = 0.0
        train_preds = []
        train_labels_list = []

        valid_loss = 0.0
        val_preds = []
        val_labels_list = []

        #train
        for i, batch in enumerate(train_data_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            images = batch['image'].to(device)
            labels = batch['label'].to(device).long()

            optimizer.zero_grad()

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, images=images).squeeze()

            loss = loss_criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            train_loss += loss.item() * input_ids.size(0)

            #"versão" comum do código, usando threshold de 0.5
            _, preds = torch.max(outputs, 1)
            preds = preds.cpu().numpy()
            labels_np = labels.cpu().numpy()

            train_preds.extend(preds)
            train_labels_list.extend(labels_np)

            
            # "versão" do código com o threshold definido
            # probs = torch.softmax(outputs, dim=1)[:, 1]  
            # preds = (probs >= threshold).long().cpu().numpy()

            # labels_np = labels.cpu().numpy()
            # val_preds.extend(preds)
            # val_labels_list.extend(labels_np)

        avg_train_loss = train_loss / train_data_size
        train_accuracy = accuracy_score(train_labels_list, train_preds)
        train_recall = recall_score(train_labels_list, train_preds)
        train_precision = precision_score(train_labels_list, train_preds)
        train_f1 = f1_score(train_labels_list, train_preds)

        #val
        with torch.no_grad():
            model.eval()

            for j, batch in enumerate(val_data_loader):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                images = batch['image'].to(device)
                labels = batch['label'].to(device).long()

                outputs = model(input_ids=input_ids, attention_mask=attention_mask, images=images).squeeze()

                loss = loss_criterion(outputs, labels)

                valid_loss += loss.item() * input_ids.size(0)

                _, preds = torch.max(outputs, 1)
                preds = preds.cpu().numpy()
                labels_np = labels.cpu().numpy()

                val_preds.extend(preds)
                val_labels_list.extend(labels_np)

            avg_valid_loss = valid_loss / val_data_size
            valid_accuracy = accuracy_score(val_labels_list, val_preds)
            valid_recall = recall_score(val_labels_list, val_preds)
            valid_precision = precision_score(val_labels_list, val_preds)
            valid_f1 = f1_score(val_labels_list, val_preds)

        if avg_valid_loss < best_loss:
            best_loss = avg_valid_loss
            best_epoch = epoch

        history.append([avg_train_loss, avg_valid_loss, train_accuracy, valid_accuracy, train_f1, valid_f1])

        epoch_end = time.time()

        print(f"Epoch : {epoch:03d}, Training: Loss - {avg_train_loss:.4f}, Accuracy - {train_accuracy * 100:.2f}%, Precision - {train_precision:.4f}, Recall - {train_recall:.4f}, F1 - {train_f1:.4f} \n\t\t"
              f"Validation : Loss - {avg_valid_loss:.4f}, Accuracy - {valid_accuracy * 100:.2f}%, Precision - {valid_precision:.4f}, Recall - {valid_recall:.4f}, F1 - {valid_f1:.4f}, Time: {epoch_end - epoch_start:.4f}s")
        torch.save(model, dataset + '_model_' + str(epoch) + '.pt')

    return model, history, best_epoch

In [26]:
#computa a performance no dataset de teste
def computeTestSetPerformance_multimodal(model, loss_criterion, test_data_loader=test_data_loader, 
                                         test_data_size=len(test_dataset), device=device):

    test_acc = 0.0
    test_loss = 0.0
    test_true_positives = 0.0
    test_false_negatives = 0.0

    with torch.no_grad():
        model.eval()

        for j, batch in enumerate(test_data_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            images = batch['image'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, images=images)

            loss = loss_criterion(outputs, labels)
            test_loss += loss.item() * input_ids.size(0)

            #_, predictions = torch.max(outputs.data, 1) #de quando experimentamos o threshold de 0.5
            probs = torch.softmax(outputs, dim=1)[:, 1] 
            predictions = (probs >= threshold).long()

            correct_counts = predictions.eq(labels.data.view_as(predictions))
            acc = correct_counts.sum().item() / input_ids.size(0)
            test_acc += acc * input_ids.size(0)

            test_true_positives += ((predictions == 1) & (labels == 1)).sum().item()
            test_false_negatives += ((predictions == 0) & (labels == 1)).sum().item()

           
    avg_test_loss = test_loss / test_data_size
    avg_test_acc = test_acc / test_data_size
    test_recall = test_true_positives / (test_true_positives + test_false_negatives + 1e-7)

    print("Test Loss: {:.4f}, Test Accuracy: {:.4f}%, Test Recall: {:.4f}".format(avg_test_loss, avg_test_acc * 100, test_recall))

    return avg_test_loss, avg_test_acc, test_recall

In [27]:
#faz a predição de uma instância em particular
def predict_multimodal(model, test_image_name, clinical_note, tokenizer, device):

    #pré-processamento da imagem de acordo com o treino
    transform = image_transforms['test']
    test_image = Image.open(test_image_name).convert('RGB')
    plt.imshow(test_image)
    test_image_tensor = transform(test_image).unsqueeze(0).to(device)

    #pré-processamento do texto
    encoded = tokenizer(clinical_note, padding='max_length', truncation=True, max_length=512, return_tensors='pt')
    input_ids = encoded['input_ids'].to(device)
    attention_mask = encoded['attention_mask'].to(device)

    #passa pelas redes
    with torch.no_grad():
        model.eval()
        out = model(input_ids=input_ids, attention_mask=attention_mask, images=test_image_tensor)

        #não precisamos usar ela com threshold diferente do default, então não mudamos aqui
        topk, topclass = out.topk(1, dim=1)
        cls = idx_to_class[topclass.cpu().numpy()[0][0]]
        score = topk.cpu().numpy()[0][0]

        print("Prediction:", cls, ", with probability:", score)

    return cls, score


In [28]:
#construindo classe do classificador 
class MultimodalNet(nn.Module):
    def __init__(self):
        super(MultimodalNet, self).__init__()

        #texto - BlueBERT
        self.text_model = AutoModel.from_pretrained('bionlp/bluebert_pubmed_mimic_uncased_L-12_H-768_A-12')
        for param in self.text_model.parameters():
            param.requires_grad = False  #feature extraction
        self.text_fc = nn.Linear(768, 256)

        #imagem - DenseNet121 pré treinada do torchxray vision
        self.image_model = xrv.models.DenseNet(weights="densenet121-res224-all")
        for param in self.image_model.parameters():
            param.requires_grad = False  #só transfer learning
        
        self.image_fc = nn.Linear(1024, 256)

        #fully connected
        self.fc_combined = nn.Sequential(
            nn.Linear(512, 128), 
            nn.LayerNorm(128),  #porque usei layernorm e não batchnorm? explicação no final
            nn.ReLU(),
            nn.Dropout(0.55),

            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.60),

            nn.Linear(64, 2)
        )

    def forward(self, input_ids, attention_mask, images):
        text_outputs = self.text_model(input_ids=input_ids, attention_mask=attention_mask)
        text_embedding = text_outputs.last_hidden_state[:, 0, :]  #pega o output do embedding (vetor denso), antes de ele entrar nas camadas de classificação do BlueBERT
        text_features = self.text_fc(text_embedding) #passa ele pra fully connected dessa rede

        image_features = self.image_model.features(images) #pega o output das convoluções, ou seja, antes de entrar nas fc layers da densenet e fazer a classificação
        image_features = image_features.view(image_features.size(0), -1)
        image_features = self.image_fc(image_features) #manda esse output para a fully connected que construí ali em cima

        #concatenação
        combined = torch.cat((text_features, image_features), dim=1)
        output = self.fc_combined(combined) 

        return output

#explicação: sem usar nenhum tipo de normalização, a rede não aprendia muito: continuava naquilo de prever a classe majoritária sempre
#como já tínhamos visto que a rede de texto funcionava bem, presumimos que as imagens estivessem "confundindo a rede" 
#- até por isso, tentamos o fine tuning da densenet
#quando experimentávamos utilizar batch normalization, a rede overfitava muito fácil. 
#pelo que pesquisamos e buscamos entender, o que aconteceu foi: utilizamos batchs pequenos 
#(apesar de grandes pro nosso dataset, pois nossa quantidade de exemplos disponíveis era pequena)
#isso atrapalhava a generalização, já que o batch norm depende muito do tamanho do batch
#usar batchs maiores não era uma opção, porque cada batch já representava mais de 10% do dataset todo
#então, procuramos por opções intermediárias, e descobrimos o layer normalization, que normaliza por instância ao longo das features
#e, portanto, não dependia do batch. Experimentamos essa abordagem, e obtivemos resultados muito bons de cara. 

In [29]:
model = MultimodalNet()
model.to(device)

MultimodalNet(
  (text_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, eleme

In [30]:
#tentamos um fine tuning da densenet aqui, por motivos já explicados
#mas não deu certo, não melhorou a performance
# for name, param in model.image_model.features.named_parameters():
#     if "denseblock4" in name or "norm5" in name:
#         param.requires_grad = True

In [31]:
boost = 2.4
counter = Counter(train_lab)
weight = (counter[0] / counter[1]) * boost

In [32]:
print(weight)

1.3528183716075157


In [33]:
#para os casos que tentamos fazer fine tuning da densenet, 
#usamos parametros distintos pras camadas dela a serem treinadas
# image_params = []
# other_params = []

# for name, param in model.named_parameters():
#     if param.requires_grad:
#         if "image_model.features.denseblock4" in name or "image_model.features.norm5" in name:
#             image_params.append(param)
#         else:
#             other_params.append(param)

In [45]:
criterion = nn.CrossEntropyLoss(weight=torch.tensor([weight, 0.91]).to(device))
# optimizer = torch.optim.AdamW([{'params': other_params, 'lr': 59e-7},{'params': image_params, 'lr': 51e-6}],
#                               weight_decay=1e-4)
optimizer = torch.optim.AdamW([p for p in model.parameters() if p.requires_grad], lr=37e-6, weight_decay=1e-4)

In [46]:
trained_model, history, best_epoch = train_and_validate_multimodal(
    model=model,
    loss_criterion=criterion,
    optimizer=optimizer,
    epochs=13
)

Epoch : 000, Training: Loss - 0.2534, Accuracy - 90.79%, Precision - 0.9399, Recall - 0.9144, F1 - 0.9270 
		Validation : Loss - 0.2447, Accuracy - 90.65%, Precision - 0.9531, Recall - 0.8971, F1 - 0.9242, Time: 205.9964s
Epoch : 001, Training: Loss - 0.2619, Accuracy - 91.19%, Precision - 0.9172, Recall - 0.9478, F1 - 0.9322 
		Validation : Loss - 0.2618, Accuracy - 90.65%, Precision - 0.9677, Recall - 0.8824, F1 - 0.9231, Time: 203.5373s
Epoch : 002, Training: Loss - 0.2333, Accuracy - 92.12%, Precision - 0.9526, Recall - 0.9228, F1 - 0.9374 
		Validation : Loss - 0.2304, Accuracy - 91.59%, Precision - 0.9683, Recall - 0.8971, F1 - 0.9313, Time: 201.5824s
Epoch : 003, Training: Loss - 0.2276, Accuracy - 93.59%, Precision - 0.9499, Recall - 0.9499, F1 - 0.9499 
		Validation : Loss - 0.2654, Accuracy - 89.72%, Precision - 0.9672, Recall - 0.8676, F1 - 0.9147, Time: 203.0049s
Epoch : 004, Training: Loss - 0.2370, Accuracy - 92.66%, Precision - 0.9380, Recall - 0.9478, F1 - 0.9429 
		Val

In [47]:
computeTestSetPerformance_multimodal(model, criterion)

Test Loss: 0.2063, Test Accuracy: 92.0561%, Test Recall: 0.8905


(0.20630931659279583, 0.9205607476635514, 0.8905109482551016)

In [1]:
history = np.array(history)
plt.plot(history[:,0:2])
plt.legend(['Train Loss', 'Validation Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim(0.3, 0.4)
plt.show()

NameError: name 'np' is not defined

In [ ]:
plt.plot(history[:,2:4])
plt.legend(['Train Accuracy', 'Validation Accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

In [44]:
torch.save(model.state_dict(), 'multimodal_model_toupdate_sintectic_data_augmentation.pth')